## Traduccion automatica usando transformers

Ajuste de parametros

In [1]:
def obtener_secuencia_maxima(archivo):
    #cadena_max_palabras = ""
    cantidad_max_palabras = 0

    with open(archivo, "r",encoding="utf-8") as archivo_txt:
      # Lee el archivo línea por línea
      for linea in archivo_txt:
        # Elimina los espacios en blanco al principio y al final de la línea
        linea = linea.strip()
        
        # Divide la línea en palabras usando los espacios como delimitadores
        palabras = linea.split()
        
        # Comprueba si la cantidad de palabras en la línea actual es mayor que la máxima cantidad actual
        if len(palabras) > cantidad_max_palabras:
            #cadena_max_palabras = linea
            cantidad_max_palabras = len(palabras)
    
    return cantidad_max_palabras    

In [2]:
def obtener_vocabulario(archivo):
    
    with open(archivo, 'r',encoding="utf-8") as f:
        # Leer el contenido del archivo
        contenido = f.read()

    # Dividir el contenido en palabras individuales
    palabras = contenido.split()

    # Crear un conjunto para almacenar las palabras únicas
    palabras_unicas = set(palabras)

    # Contar las palabras únicas
    contador = len(palabras_unicas)

    return contador

In [3]:
def definir_secuencia(archivo1,archivo2,archivo3,limite_palabras):
    lineas_validas = []

    with open(archivo1, "r",encoding="utf-8") as archivo1_txt, open(archivo2, "r",encoding="utf-8") as archivo2_txt, open(archivo3, "r",encoding="utf-8") as archivo3_txt:
      # Lee ambos archivos línea por línea simultáneamente
      for linea1, linea2, linea3 in zip(archivo1_txt, archivo2_txt, archivo3_txt):
        # Elimina los espacios en blanco al principio y al final de las líneas
        linea1 = linea1.strip()
        linea2 = linea2.strip()
        linea3 = linea3.strip()
        
        # Divide las líneas en palabras usando los espacios como delimitadores
        palabras1 = linea1.split()
        palabras2 = linea2.split()
        palabras3 = linea3.split()
        
        # Comprueba si ambas líneas cumplen con el límite de palabras
        if len(palabras1) <= limite_palabras and len(palabras2) <= limite_palabras and len(palabras3) <= limite_palabras:
            lineas_validas.append((linea1, linea2, linea3))

    # Sobreescribe ambos archivos con las líneas válidas
    with open(archivo1, "w",encoding="utf-8") as archivo1_txt, open(archivo2, "w",encoding="utf-8") as archivo2_txt, open(archivo3, "w",encoding="utf-8") as archivo3_txt:
        for linea1, linea2, linea3 in lineas_validas:
            archivo1_txt.write(linea1 + "\n")
            archivo2_txt.write(linea2 + "\n")
            archivo3_txt.write(linea3 + "\n")

## Recoleccion de datos
**Datos de entrenamiento**

In [73]:
train_pairs = []

text_file1 = "Data/train1Es.txt" 
text_file2 = "Data/train1Qu.txt"
text_file3 = "Data/train1Ai.txt"
definir_secuencia(text_file1,text_file2, text_file3,20)
vocab_size_esp = obtener_vocabulario(text_file1)
vocab_size_que = obtener_vocabulario(text_file2) 
vocab_size_ai = obtener_vocabulario(text_file3) 
sequence_length = 20

with open(text_file1,encoding="utf-8") as f:
    lines1 = f.read().split("\n")[:-1]
with open(text_file2,encoding="utf-8") as f:
    lines2 = f.read().split("\n")[:-1]
with open(text_file3,encoding="utf-8") as f:
    lines3 = f.read().split("\n")[:-1]

for i in range(0,len(lines2)):
    español  = lines1[i]
    quechua  = lines2[i]
    if español != quechua:
       quechua = "[start] " + quechua + " [end]"
       train_pairs.append((español, quechua))
"""    

for i in range(0,len(lines1)):
    español  = lines1[i]
    aimara  = lines3[i]
    if español != aimara:
       español = "[start] " + español + " [end]"
       train_pairs.append((aimara, español))
"""       

'    \n\nfor i in range(0,len(lines1)):\n    español  = lines1[i]\n    aimara  = lines3[i]\n    if español != aimara:\n       español = "[start] " + español + " [end]"\n       train_pairs.append((aimara, español))\n'

In [74]:
obtener_secuencia_maxima(text_file3)

20

**Datos de validacion**

In [2]:
text_file1 = "Data/validationEs.txt" 
text_file2 = "Data/validationQu.txt"

with open(text_file1,encoding="utf-8") as f:
    lines1 = f.read().split("\n")[:-1]
with open(text_file2,encoding="utf-8") as f:
    lines2 = f.read().split("\n")[:-1]
##text_pairs = [] 
val_pairs = []

for i in range(0,len(lines2)):
    español  = lines1[i]
    quechua  = lines2[i]
    if español != quechua:
       quechua = "[start] " + quechua + " [end]"
       ##text_pairs.append((español, quechua))
       val_pairs.append((español, quechua))

**Datos de prueba**

In [3]:
text_file1 = "Data/testEs.txt" 
text_file2 = "Data/testQu.txt"

with open(text_file1,encoding="utf-8") as f:
    lines1 = f.read().split("\n")[:-1]
with open(text_file2,encoding="utf-8") as f:
    lines2 = f.read().split("\n")[:-1]
##text_pairs = [] 
test_pairs = []

for i in range(0,len(lines2)):
    español  = lines1[i]
    quechua  = lines2[i]
    if español != quechua:
       quechua = "[start] " + quechua + " [end]"
       ##text_pairs.append((español, quechua))
       test_pairs.append((español, quechua))

Ejemplo de los datos

In [75]:
import random
print(random.choice(train_pairs))

('Nos consolará y dará fuerzas cuando nos enfrentemos a pruebas severas o tengamos alguna necesidad especial.', '[start] Chayqa sonqochawasunmi kallpata qowasun sinchi pruebakunapi tarikuspa otaq necesidad especialkunapi tarikuspanchis. [end]')


**Vectorizando los pares de texto en español y quechua**

In [76]:
import tensorflow as tf
from tensorflow.keras import layers
import string
import re

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

##Convierte las letras mayusculas en minusculas
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string) 
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

##vocab_size = 32935   #Tamaño máximo del vocabulario para esta capa.
##sequence_length = 20  #dimension de la salida

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size_esp,
    output_mode="int",
    output_sequence_length=sequence_length,
)

target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size_que,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
 #  standardize="lower_and_strip_punctuation",
    standardize=custom_standardization,
)
"""
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size_ai,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
 #  standardize="lower_and_strip_punctuation",
    standardize=custom_standardization,
)
"""
train_spanish_texts = [pair[0] for pair in train_pairs]
train_quechua_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_spanish_texts)
target_vectorization.adapt(train_quechua_texts)

**Preparando conjuntos de datos para la tarea de traducción**

In [77]:
batch_size = 100

def format_dataset(spa, que):
    spa = source_vectorization(spa)
    que = target_vectorization(que)
    return ({
        "español": spa,
        "quechua": que[:, :-1],
    }, que[:, 1:])

def make_dataset(pairs):
    spa_texts, que_texts = zip(*pairs)
    spa_texts = list(spa_texts)
    que_texts = list(que_texts)
    #Representa un conjunto potencialmente grande de elementos.
    dataset = tf.data.Dataset.from_tensor_slices((spa_texts, que_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
#val_ds = make_dataset(val_pairs)
#test_ds = make_dataset(test_pairs)

for inputs, targets in train_ds.take(1):
    print(f"inputs['español'].shape: {inputs['español'].shape}")
    print(f"inputs['quechua'].shape: {inputs['quechua'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['español'].shape: (100, 20)
inputs['quechua'].shape: (100, 20)
targets.shape: (100, 20)


### Modelo de traduccion usando redes Transformers

**Codificador posicional**

In [64]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**Codificador**

In [65]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([layers.Dense(dense_dim, activation="relu"),layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

**Decodificador**

In [66]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([layers.Dense(dense_dim, activation="relu"),layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat([tf.expand_dims(batch_size, -1),tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(query=inputs,value=inputs,key=inputs,attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(query=attention_output_1,value=encoder_outputs,key=encoder_outputs,attention_mask=padding_mask,)
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

**Procesamiento del modelo**

In [67]:
import keras
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="español")
x = PositionalEmbedding(sequence_length, vocab_size_ai, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="quechua")
x = PositionalEmbedding(sequence_length, vocab_size_esp, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)

decoder_outputs = layers.Dense(vocab_size_esp, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

**Datos de entramiento**

In [68]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
#transformer.fit(train_ds, epochs=30, validation_data=val_ds)
transformer.fit(train_ds, epochs=40)

Epoch 1/40
76/76 [==============================] - 186s 2s/step - loss: 6.4477 - accuracy: 0.1643
Epoch 2/40
76/76 [==============================] - 195s 3s/step - loss: 5.5022 - accuracy: 0.2214
Epoch 3/40
76/76 [==============================] - 199s 3s/step - loss: 5.1216 - accuracy: 0.2528
Epoch 4/40
76/76 [==============================] - 191s 3s/step - loss: 4.8304 - accuracy: 0.2807
Epoch 5/40
76/76 [==============================] - 207s 3s/step - loss: 4.5523 - accuracy: 0.3049
Epoch 6/40
76/76 [==============================] - 213s 3s/step - loss: 4.3116 - accuracy: 0.3308
Epoch 7/40
76/76 [==============================] - 209s 3s/step - loss: 4.0769 - accuracy: 0.3529
Epoch 8/40
76/76 [==============================] - 201s 3s/step - loss: 3.8545 - accuracy: 0.3790
Epoch 9/40
76/76 [==============================] - 208s 3s/step - loss: 3.6168 - accuracy: 0.4077
Epoch 10/40
76/76 [==============================] - 200s 3s/step - loss: 3.4130 - accuracy: 0.4312
Epoch 11/

In [85]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 30

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        ##predictions = new_model([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_spa_texts = [pair[0] for pair in train_pairs]
test_que_texts = [pair[1] for pair in train_pairs]
traduccion = []
original = []
for i in range(0,10):
    input_sentence = test_spa_texts[i]
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))
    original.append(test_que_texts[i])
    traduccion.append(decode_sequence(input_sentence))

-
Y por último dice que el pueblo de Dios “escapará, todo el que se halle escrito en el libro ”.
[start] hinaspa tukunanpaq nin diospa llaqtanqa ayqenqakun llapa libropi qelqasqa tarisqakuna [end]
-
b) ¿Cómo lograban mantenerse estos siervos de tiempo completo?
[start] b imaynatan wakinqa historiapa puriynintam cambiarunku [end]
-
Pero al mismo tiempo nos advierte que “el amor al dinero es raíz de toda clase de males ”.
[start] ichaqa chay pachallapitaqmi anyawanchis qolqe munakuyqa tukuy imaymana mana allinkunaq saphinmi [end]
-
RESPUESTA: El Reino de Dios es un gobierno celestial, y su Rey es Jesús.
[start] kutichiy diospa reinonqa hanaq pacha gobiernon reynintaq jesus [end]
-
¿CUÁL ES LA SOLUCIÓN?
[start] imataq allinchay [end]
-
Adán y Eva desobedecieron a Dios, de modo que fueron expulsados del Edén.
[start] adanwan evawanqa manan diosta kasukurqankuchu chaymi edén llaqtamanta qarqosqa karqanku [end]
-
Jesús cumplió de forma sorprendente esta profecía durante su ministerio.
[start

**Guarda el modelo entrenado**

Una vez que hallamos terminado con nuestras pruebas podemos guardar todo los parametros de nuestro modelo para poder utilizarlo mas adelante.

In [86]:
original

['[start] Hinaspa tukunanpaq nin Diospa llaqtanqa "ayqenqakun, llapa libropi qelqasqa tarisqakuna". [end]',
 '[start] b)\xa0¿Imaynatam tukuy tiemponkuwan serviqkunaqa yanapakurqaku? [end]',
 '[start] Ichaqa chay pachallapitaqmi anyawanchis "qolqe munakuyqa tukuy imaymana mana allinkunaq saphinmi". [end]',
 '[start] KUTICHIY: Diospa Reinonqa hanaq pacha gobiernon, Reynintaq Jesus. [end]',
 '[start] ¿IMATAQ ALLINCHAY? [end]',
 '[start] Adanwan Evawanqa manan Diosta kasukurqankuchu, chaymi Edén llaqtamanta qarqosqa karqanku. [end]',
 '[start] Jesusqa admirakunapaq hinan hunt’arqan predicashaspa chay profeciata. [end]',
 '[start] Yachasqanchik yachachikuykunata [end]',
 '[start] Hinaspapas chay mana kaqlla kasqankuqa huk cheqasllapi tiyaq runakuna ukhupipas rikukunmi. [end]',
 '[start] • ¿Imaynatan Asaf, Jeremias, David ima rikuchirqanku Jehová Dios herencianku kasqanta? [end]']

In [70]:
transformer.save('modeloAimaraEspañol.h5')

**Instalar un modelo de traduccion ya entrenado**

para este caso no se debera realizar el procesamiento del modelo con los datos de entrenamiento y pasarse de frente a las pruebas. 

In [71]:
import keras
transformer = keras.models.load_model('modeloEspañolQuechua.h5',custom_objects={'PositionalEmbedding': PositionalEmbedding,'TransformerEncoder': TransformerEncoder,'TransformerDecoder':TransformerDecoder})